In [3]:
from neo4j import GraphDatabase
import re
advent = "Day2"
workdir = 'C:\\Users\\ed.deane\\OneDrive - Sabio Ltd\\Documents\\Customers\\IKEA\\Knowledge\\Python\\AdventOfCode\\2023\\'


f = open(workdir+advent+"\\input_1.txt", "r")
games = []
cubes = {"red":{"max_val":12},"green":{"max_val":13}, "blue":{"max_val":14}}
         
green_cubes = 13  
blue_cubes = 14

for line in f:
    line = line.strip()
    print (line)
    games.append(line)

#create a node for each Game (id=n)
uri = "bolt://localhost:7687"
username = "neo4j"
password = "advent_of_code"
# Cypher query to create a node with the index as the node name and value as the 'prop' property
query_del = "MATCH (n) DETACH DELETE n"

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
            session.run(query_del)


query_game = "CREATE (:game {name: $game_num})"
query_cube = "CREATE (:cube {name: $cube_name, max_val: $max_val})"
query_turn = """
            MATCH (g:game {name: $game_num})
            MATCH (c:cube {name: $cube_name})
            CREATE (g)-[:turn  {cube_total:$cube_total, turn_number:$turn_number}]->(c)
            """

def create_game_node(tx, game_num):
    tx.run(query_game, game_num=game_num)

def create_cube_node(tx, cube_name, max_val):
    tx.run(query_cube, cube_name=cube_name, max_val=max_val)
    
def create_turn_relationship(tx, cube_name, game_num, turn_number, cube_total):
    print (cube_name, game_num, turn_number, cube_total)
    tx.run(query_turn, game_num=game_num, cube_name=cube_name, turn_number=turn_number, cube_total=cube_total)

# create a node for each colour 
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    # Iterate through the list and create nodes
    
    cubes = {"red":{"max_val":12},"green":{"max_val":13}, "blue":{"max_val":14}}

    for x, y in cubes.items():
        print(x,y["max_val"])
        with driver.session() as session:
            session.write_transaction(create_cube_node,x,y["max_val"])
            
    # create a node for each game
    for i in games:
        game_num = (i.split(":")[0])
        cube_subsets = i.split(":")[1].split(';')

        # Iterate through the list and create nodes
        with driver.session() as session:
            session.write_transaction(create_game_node,game_num)

#connect each game to its colour for each go (relationship attribute 'turn' equal to 1, 2, 3)

        turn_number = 0
        for subset in cube_subsets:
            turn_number += 1
            for turn_cubes in subset.split(','):
                cube_total,cube_name = turn_cubes.split()
                print (cube_total.split(), game_num,turn_number)
                with driver.session() as session:
                    session.write_transaction(create_turn_relationship,
                                              cube_name, 
                                              game_num, 
                                              turn_number, 
                                              int(cube_total))
                                              
#query the db for all nodes that only contain relationships below the threshold

"""
Test 1

MATCH (game:game)-[turn:turn]-(cube:cube)
WHERE turn.cube_total > cube.max_val
WITH collect(distinct game) as games_collected
MATCH (game:game)
WHERE NOT game IN games_collected
RETURN sum(toInteger(apoc.text.regexGroups(game.name, 'Game (\\d+)')[0][1])) as gameNumber

Test 2
MATCH (game:game)-[turn:turn]-(cube:cube)
WITH game, cube, MAX(turn.cube_total) AS max_cube_total
ORDER BY game, cube
WITH game, COLLECT(max_cube_total) AS max_cube_totals
RETURN
  sum(REDUCE(s = 1, max_cube_total IN max_cube_totals | s * max_cube_total)) AS multiplied_max_cube_total;

"""

Game 1: 1 red, 3 blue, 11 green; 1 blue, 5 red; 3 blue, 5 green, 13 red; 6 red, 1 blue, 4 green; 16 red, 12 green
Game 2: 3 red, 13 blue, 5 green; 14 green, 14 blue; 9 blue, 10 green, 3 red; 2 green, 5 blue; 11 green, 3 blue, 3 red; 16 blue, 2 red, 9 green
Game 3: 17 blue, 5 red; 3 red, 11 green, 17 blue; 1 red, 6 blue, 9 green; 3 blue, 11 green, 1 red; 3 green, 10 red, 11 blue; 12 red, 3 green, 15 blue
Game 4: 14 green, 14 red, 1 blue; 15 red, 13 green, 1 blue; 6 green, 15 red; 7 green
Game 5: 3 green, 1 blue, 3 red; 6 red, 2 green, 2 blue; 12 red, 3 green, 1 blue; 2 green, 9 red; 1 blue; 2 blue, 10 red
Game 6: 5 blue, 5 green; 4 blue, 1 red, 10 green; 16 green, 1 red, 6 blue; 1 red, 1 blue, 13 green; 1 red, 5 blue, 7 green; 14 green, 17 blue
Game 7: 1 green, 8 blue, 4 red; 1 green, 4 blue, 4 red; 6 blue, 4 red, 4 green; 1 red, 8 green
Game 8: 2 red, 5 blue, 1 green; 1 blue, 4 red, 8 green; 6 blue, 12 green, 6 red; 3 blue, 5 red; 8 red, 2 blue, 13 green; 5 green, 4 red, 3 blue
Game 9:

red 12


C:\Users\ed.deane\AppData\Local\Temp\ipykernel_21876\3152364821.py:58: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_cube_node,x,y["max_val"])


green 13
blue 14
['1'] Game 1 1
red Game 1 1 1
['3'] Game 1 1
blue Game 1 1 3
['11'] Game 1 1
green Game 1 1 11
['1'] Game 1 2
blue Game 1 2 1
['5'] Game 1 2
red Game 1 2 5
['3'] Game 1 3
blue Game 1 3 3
['5'] Game 1 3
green Game 1 3 5
['13'] Game 1 3
red Game 1 3 13
['6'] Game 1 4
red Game 1 4 6
['1'] Game 1 4
blue Game 1 4 1
['4'] Game 1 4
green Game 1 4 4
['16'] Game 1 5
red Game 1 5 16
['12'] Game 1 5
green Game 1 5 12
['3'] Game 2 1
red Game 2 1 3
['13'] Game 2 1
blue Game 2 1 13
['5'] Game 2 1
green Game 2 1 5
['14'] Game 2 2
green Game 2 2 14
['14'] Game 2 2
blue Game 2 2 14
['9'] Game 2 3
blue Game 2 3 9
['10'] Game 2 3
green Game 2 3 10
['3'] Game 2 3
red Game 2 3 3
['2'] Game 2 4
green Game 2 4 2
['5'] Game 2 4
blue Game 2 4 5
['11'] Game 2 5
green Game 2 5 11
['3'] Game 2 5
blue Game 2 5 3
['3'] Game 2 5
red Game 2 5 3
['16'] Game 2 6
blue Game 2 6 16
['2'] Game 2 6
red Game 2 6 2
['9'] Game 2 6
green Game 2 6 9
['17'] Game 3 1
blue Game 3 1 17
['5'] Game 3 1
red Game 3 1 5


C:\Users\ed.deane\AppData\Local\Temp\ipykernel_21876\3152364821.py:67: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_game_node,game_num)
C:\Users\ed.deane\AppData\Local\Temp\ipykernel_21876\3152364821.py:78: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_turn_relationship,


['1'] Game 6 5
red Game 6 5 1
['5'] Game 6 5
blue Game 6 5 5
['7'] Game 6 5
green Game 6 5 7
['14'] Game 6 6
green Game 6 6 14
['17'] Game 6 6
blue Game 6 6 17
['1'] Game 7 1
green Game 7 1 1
['8'] Game 7 1
blue Game 7 1 8
['4'] Game 7 1
red Game 7 1 4
['1'] Game 7 2
green Game 7 2 1
['4'] Game 7 2
blue Game 7 2 4
['4'] Game 7 2
red Game 7 2 4
['6'] Game 7 3
blue Game 7 3 6
['4'] Game 7 3
red Game 7 3 4
['4'] Game 7 3
green Game 7 3 4
['1'] Game 7 4
red Game 7 4 1
['8'] Game 7 4
green Game 7 4 8
['2'] Game 8 1
red Game 8 1 2
['5'] Game 8 1
blue Game 8 1 5
['1'] Game 8 1
green Game 8 1 1
['1'] Game 8 2
blue Game 8 2 1
['4'] Game 8 2
red Game 8 2 4
['8'] Game 8 2
green Game 8 2 8
['6'] Game 8 3
blue Game 8 3 6
['12'] Game 8 3
green Game 8 3 12
['6'] Game 8 3
red Game 8 3 6
['3'] Game 8 4
blue Game 8 4 3
['5'] Game 8 4
red Game 8 4 5
['8'] Game 8 5
red Game 8 5 8
['2'] Game 8 5
blue Game 8 5 2
['13'] Game 8 5
green Game 8 5 13
['5'] Game 8 6
green Game 8 6 5
['4'] Game 8 6
red Game 8 6 4


['18'] Game 29 4
blue Game 29 4 18
['3'] Game 29 4
red Game 29 4 3
['16'] Game 29 5
green Game 29 5 16
['10'] Game 29 5
red Game 29 5 10
['16'] Game 29 6
green Game 29 6 16
['12'] Game 29 6
blue Game 29 6 12
['16'] Game 29 6
red Game 29 6 16
['14'] Game 30 1
blue Game 30 1 14
['4'] Game 30 1
green Game 30 1 4
['1'] Game 30 1
red Game 30 1 1
['7'] Game 30 2
red Game 30 2 7
['14'] Game 30 2
blue Game 30 2 14
['2'] Game 30 3
blue Game 30 3 2
['4'] Game 30 3
red Game 30 3 4
['1'] Game 30 3
green Game 30 3 1
['2'] Game 31 1
red Game 31 1 2
['14'] Game 31 1
green Game 31 1 14
['3'] Game 31 1
blue Game 31 1 3
['3'] Game 31 2
blue Game 31 2 3
['3'] Game 31 2
green Game 31 2 3
['4'] Game 31 2
red Game 31 2 4
['8'] Game 31 3
blue Game 31 3 8
['4'] Game 31 3
red Game 31 3 4
['1'] Game 31 3
green Game 31 3 1
['8'] Game 31 4
green Game 31 4 8
['3'] Game 31 4
blue Game 31 4 3
['10'] Game 31 5
blue Game 31 5 10
['1'] Game 31 5
red Game 31 5 1
['11'] Game 31 5
green Game 31 5 11
['13'] Game 31 6
green

['2'] Game 54 1
red Game 54 1 2
['8'] Game 54 1
blue Game 54 1 8
['15'] Game 54 1
green Game 54 1 15
['4'] Game 54 2
green Game 54 2 4
['3'] Game 54 2
blue Game 54 2 3
['6'] Game 54 2
red Game 54 2 6
['12'] Game 54 3
green Game 54 3 12
['13'] Game 54 3
blue Game 54 3 13
['4'] Game 54 3
red Game 54 3 4
['1'] Game 55 1
green Game 55 1 1
['16'] Game 55 1
blue Game 55 1 16
['4'] Game 55 1
red Game 55 1 4
['3'] Game 55 2
red Game 55 2 3
['1'] Game 55 2
blue Game 55 2 1
['1'] Game 55 2
green Game 55 2 1
['12'] Game 55 3
red Game 55 3 12
['16'] Game 55 3
blue Game 55 3 16
['3'] Game 55 4
red Game 55 4 3
['4'] Game 56 1
green Game 56 1 4
['1'] Game 56 2
red Game 56 2 1
['4'] Game 56 2
green Game 56 2 4
['2'] Game 56 3
red Game 56 3 2
['3'] Game 56 3
blue Game 56 3 3
['7'] Game 56 3
green Game 56 3 7
['2'] Game 56 4
red Game 56 4 2
['3'] Game 56 4
blue Game 56 4 3
['15'] Game 56 4
green Game 56 4 15
['17'] Game 57 1
green Game 57 1 17
['1'] Game 57 2
green Game 57 2 1
['9'] Game 57 2
blue Game 

['3'] Game 81 2
green Game 81 2 3
['1'] Game 81 2
blue Game 81 2 1
['6'] Game 81 3
blue Game 81 3 6
['1'] Game 81 3
green Game 81 3 1
['3'] Game 81 4
blue Game 81 4 3
['3'] Game 81 4
green Game 81 4 3
['1'] Game 81 4
red Game 81 4 1
['2'] Game 81 5
green Game 81 5 2
['8'] Game 81 5
blue Game 81 5 8
['1'] Game 81 6
red Game 81 6 1
['8'] Game 81 6
blue Game 81 6 8
['2'] Game 81 6
green Game 81 6 2
['5'] Game 82 1
blue Game 82 1 5
['4'] Game 82 1
red Game 82 1 4
['1'] Game 82 1
green Game 82 1 1
['9'] Game 82 2
red Game 82 2 9
['12'] Game 82 2
green Game 82 2 12
['8'] Game 82 2
blue Game 82 2 8
['9'] Game 82 3
red Game 82 3 9
['6'] Game 82 3
green Game 82 3 6
['15'] Game 82 3
blue Game 82 3 15
['8'] Game 82 4
blue Game 82 4 8
['10'] Game 82 4
red Game 82 4 10
['6'] Game 82 4
green Game 82 4 6
['2'] Game 83 1
green Game 83 1 2
['7'] Game 83 1
blue Game 83 1 7
['4'] Game 83 1
red Game 83 1 4
['2'] Game 83 2
blue Game 83 2 2
['11'] Game 83 2
red Game 83 2 11
['9'] Game 83 2
green Game 83 2 9

"\nMATCH (game:game)-[turn:turn]-(cube:cube)\nWHERE turn.cube_total > cube.max_val\nWITH collect(distinct game) as games_collected\nMATCH (game:game)\nWHERE NOT game IN games_collected\nRETURN sum(toInteger(apoc.text.regexGroups(game.name, 'Game (\\d+)')[0][1])) as gameNumber\n"